# Welcome to Intro to Artificial Inteligence assignment 1 practical part!

In this part you will get to implement algotithms learned in class from scratch.


The pupose of this notebook is to provide the requirments of the practical part and some visualizations. Your implementation should be in the provided Algorithms.py file.

**Important tip:** If the same variable name appears in two different cells, the variable value will be determined by the last cell to run, rather than by the position of the cell. Let's see an example,  run the three cells below:

In [1]:
# cell 1
x = 3

In [2]:
# cell 2
x = 5

In [3]:
print(x)

5


now rerun cell 1 and print x again.

The same applies to functions, classes, etc.


# The Environment


1. Your goal is to find a route from the initial state to **a** goal state.
2. Each position on the map is marked with a letter, and each letter has a different meaning:

  *   S - Initial (start) position. There's only one initial state on the map and its always on the upper-left corner of the grid.
  *   G - Final state. As learned in class there may be multiple goal states.
  *   H - Hole. When your agent reaches a hole, it falls into the water and cannot continue walking.
  *   D - Dragon Ball. There are either two on the map. When the agent reaches a Dragon Ball, it is immediately collected. You can assume that they will not appear in a final or initial state.
  *   F - This is the most common square on the map. The agent can walk on it safely.

  Your agent can move faster by collecting 3 special objects.

  *   T - [Talaria](https://en.wikipedia.org/wiki/Talaria). A pair of Winged Sandals that help you fly to the square.
  *   A - Air jorden. Jumping shoes that help you jump to the square.
  *   L - Lightning. Makes you run faster to the square.

  When the agent performs transition (s,a,s') and collects one of the objects (T,A,L) at state s', the collected object changes the cost of the transition (as detailed below). 

3. The cost of each transition (s,a,s') is based on the mark if the square s' you pass to:
  * S - 1
  * G - 1
  * H - ֿ$\infty$
  * D - 1
  * F - 10
  * T - 3
  * A - 2
  * L - 1

4. The pink square marks where your agent is.

5. The agent position is calculated as follows: row_number *num_col + col_ number*.

6. Our agent can perform 4 actions:
  * 0 - Down
  * 1 - Right
  * 2 - Up
  * 3 - Left
  

7. If the agent tries to move outside the board boundaries, he stays in the same place.

8. Section 6 describes the order in which the nodes should be created.



In [4]:
import time
from IPython.display import clear_output



from DragonBallEnv import DragonBallEnv
from typing import List, Tuple
from Algorithms import *


In [5]:
DOWN = 0
RIGHT = 1
UP = 2
LEFT = 3

# Maps
A map can be produced manually as shown in the cell below. We will only work on maps in which there is a route from the initial state to a final state.

In [6]:
MAPS = {
    "4x4": ["SFFF",
            "FDFF",
            "FFFD",
            "FFFG"],
    "8x8": [
        "SFFFFFFF",
        "FFFFFTAL",
        "TFFHFFTF",
        "FFFFFHTF",
        "FAFHFFFF",
        "FHHFFFHF",
        "DFTFHDTL",
        "FLFHFFFG",
    ],
}


# The Dragon Ball Environment ⛷
The file `DragonBallEnv.py` implements our own version of the dragon ball environment. It is recommended to go through the code. 

**Note: You are not allowed to change this file.**

Lets start by creating a new environment object.

In order to create an environment object, you must provide it with a board.

In [7]:
env = DragonBallEnv(MAPS["8x8"])
state = env.reset()
print('Initial state:', state)
print('Goal states:', env.goals) 


Initial state: (0, False, False)
Goal states: [(63, True, True)]


First, take a look at the state space $\mathcal{S}$ (all possible states) and action space $\mathcal{A}$ (all possible actions). 

In [8]:
print(f"Action Space {env.action_space}")
print(f"State Space {env.observation_space}")

Action Space Discrete(4)
State Space Tuple(Discrete(64), Discrete(2), Discrete(2))


***Remark***: You may have noticed that gym uses `observation_space` instead of state space. For the purpose of this homework, the state space is the same as the observation space. However, in some problems the state is partially observed, so the space of possible states may not be the same as the space of possible observations. 



Now we will go through some usfel methods (It is still recommended to go through the other methods in the class):

`render()` - returns a printable view of the board.

In [9]:
print(env.render())
print(env.get_state())


SFFFFFFF
FFFFFTAL
TFFHFFTF
FFFFFHTF
FAFHFFFF
FHHFFFHF
DFTFHDTL
FLFHFFFG

(0, False, False)


The pink square represents the agent. The letter ״*H*״ represents holes, and the yellow square are dragon balls or goal.

Here are two more useful methods:

`get_state()` - Returns the current state of the agent.

`set_state(state)` - Sets the current state of the agent.

In [10]:
# env.set_state((18, False, False))
print(env.render())
print(f"the agent is at state: {env.get_state()}")


SFFFFFFF
FFFFFTAL
TFFHFFTF
FFFFFHTF
FAFHFFFF
FHHFFFHF
DFTFHDTL
FLFHFFFG

the agent is at state: (0, False, False)


`succ(state)` - Returns a dictionary that contains information on all the successors of a state.

*   The keys are the actions.
*   The values are tuples of the form (next state, cost, terminated). Note that terminated is true when the agent reaches a **final state** or a **hole**.



***Tip***: You can loop through both keys and values by using the `items()` method.


In [11]:
current_state = env.get_state()
print(f"Current state: {current_state}\n")
for action, successor in env.succ(current_state).items():
  print(f"*** Action: {action} ***")
  print(f"Next state: {successor[0]}")
  print(f"Cost: {successor[1]}")
  print(f"Terminated: {successor[2]}\n")

Current state: (0, False, False)

*** Action: 0 ***
Next state: (8, False, False)
Cost: 10.0
Terminated: False

*** Action: 1 ***
Next state: (1, False, False)
Cost: 10.0
Terminated: False

*** Action: 2 ***
Next state: (0, False, False)
Cost: 1.0
Terminated: False

*** Action: 3 ***
Next state: (0, False, False)
Cost: 1.0
Terminated: False


As you can see, the action 0 (down) will move your agent to state 26 and the transition will cost you 10. Action 1 will move your agent to state 19, which is a hole that will terminate your run.

`is_final_state(state)` can assist you in distinguishing between a final state and a hole.

In [12]:
state, cost, terminated = env.succ(current_state)[1]

print(f"Next state: {state}")
print(f"Cost: {cost}")
print(f"Terminated: {terminated}")
print(f"Final state: {env.is_final_state(state)}")

Next state: (1, False, False)
Cost: 10.0
Terminated: False
Final state: False


Let's see what happens when we apply succ(state) on a hole:



In [13]:
print(f"Current state: 19\n")
for action, (state, cost, terminated) in env.succ((19, False, False)).items():
  print(f"*** Action: {action} ***")
  print(f"Next state: {state}")
  print(f"Cost: {cost}")
  print(f"Terminated: {terminated}\n")

Current state: 19

*** Action: 0 ***
Next state: None
Cost: None
Terminated: None

*** Action: 1 ***
Next state: None
Cost: None
Terminated: None

*** Action: 2 ***
Next state: None
Cost: None
Terminated: None

*** Action: 3 ***
Next state: None
Cost: None
Terminated: None


As you can see, if the operator cannot be applied to the state, all returned values are "None"

Now it's time to move your agent around 🤖.

`step(action)` - will move your agent one step along the board.


In [14]:
new_state, cost, terminated = env.step(UP)
print(env.render())
print(env.to_row_col(new_state))
print("New state:", new_state)
print("cost:", cost)
print("Terminated:", terminated)

  (Up)
SFFFFFFF
FFFFFTAL
TFFHFFTF
FFFFFHTF
FAFHFFFF
FHHFFFHF
DFTFHDTL
FLFHFFFG

(0, 0)
New state: (0, False, False)
cost: 1.0
Terminated: False


The step-function returns the following information:
* __New state__: The state after the action is taken.
* __Cost__: The immediate cost.
* __Terminated__: Is the environment done? In our environment this will be false until the agent will reach a hole or a final state.


Here are a few more useful attributes and methods:


`env.nrow`, `env.ncol` - Row and columns number.

`env.nA` - Number of actions.

`env.nS` - Number of squares.

`env.lastaction` - The last action performed by the agent.

`env.d1`, `env.d2` - This is the state for each dragon ball. If there are no dragon balls on the board they are set to NULL.

`env.inc(row, col, action)` - Given a position and an action, returns the new position.

`env.to_row_col(state)` - Converts between state and location on the board.

`env.to_state(row, col)` - Converts between location on the board and state.



We've finished our demo 🥳 and it's time to reset the environment.

In [15]:
print(f"current state befor reset: {env.get_state()}")
env.reset()
print(f"current state after reset: {env.get_state()}")

current state befor reset: (0, False, False)
current state after reset: (0, False, False)


One (quite bad) strategy for out agent is to take a random action every time. Inside a gym-environment this can be done using `env.action_space.sample()`, which samples a random action from the action space. Look through the following loop and make sure that you understand what's going on. Here, we use `clear_output()` to clear the output of the Jupyter cell, and `time.sleep()` to pause between each action)


Let's see what would happen if we try to brute-force our way to solving the problem.


We'll create an infinite loop that runs until the agent reaches the final state.The `env.action_space.sample()` method automatically selects one random action from set of all possible actions.

In [16]:
class RandomAgent():
  def __init__(self):
    self.env = None

  def animation(self, epochs: int ,state: int, action: List[int], total_cost: int) -> None:
      clear_output(wait=True)
      print(self.env.render())
      print(f"Timestep: {epochs}")
      print(f"State: {state}")
      print(f"Action: {action}")
      print(f"Total Cost: {total_cost}")
      time.sleep(1)

  def random_search(self, DragonBallEnv: env) -> Tuple[List[int],int]:
    self.env = env
    self.env.reset()
    epochs = 0
    cost = 0
    total_cost = 0

    actions = []

    state = self.env.get_initial_state()
    while not self.env.is_final_state(state):
      action = self.env.action_space.sample()
      new_state, cost, terminated = self.env.step(action)
        
      while terminated is True and self.env.is_final_state(state) is False:
        self.env.set_state(state)
        action = self.env.action_space.sample()
        new_state, cost, terminated = self.env.step(action)
        
      actions.append(action)
      total_cost += cost
      state = new_state
      epochs += 1
      
      self.animation(epochs,state,action,total_cost)

    return (actions, total_cost)

Let's check out this agent's performance!

The output of this agent is the sequence of actions that led to the solution and the route's cost. 

Our random agent is not very successful, so we'll print his actions as they happen. 

1.   **Stop his run in the middle if you are tired of looking at him.**
2.   After watching the agent please put the code in the box below in the a comment for your comfort.

 



In [17]:
# agent = RandomAgent()
# agent.random_search(env)

**Did you remember to put the code above in a comment?!**

As you can see, a random policy is, unsurprisingly, not a good policy. However, what else can we do?

This is where you come in!

In this assignment you will be required to implement the following algorithms taught in class in order to solve the problem.

Algorithms: 
1. BFS-G
2. W-A*
3. epsilon-A*

Important to note!

Each agent should return a tuple: (actions, cost, expended) 
*  actions - the list of integers containing the sequence of actions that produce your agent's solution (and not the entire search process).
* cost -  an integer which holds the total cost of the solution.
* expanded - an integer which holds the number of nodes that have been expanded during the search (A node is considered expanded if we check for it's successors).

The solution to our search problem is the a to the final state, not the final state itself. By saving the actions, we are able to restore the path your agent found.


Any other output, unless otherwise specified, will cause the running of the tests to fail and will result in a grade of 0 !




Some Tips:
1. Follow the pseudo-code shown in the lectures and tutorials.
2. You should write all your code within the classes. This way, we prevent overlapping functions with the same name while running the notebook.
3. You may implement your code as you like but consider inherenting from the a general "Agent" class and implement some utilty methods such as the "solution" method which recieves a node and returns a path (sequence of actions) leading to that node.
4. Consider implementing a "node" class.
5. Using small boards will help you debug.


The function below (`print_solution()`) can be used for debugging purposes. It prints the sequence of actions it receives. The function will not be used to test your code, so you are welcome to change it.

In [18]:
def print_solution(actions,env: DragonBallEnv) -> None:
    env.reset()
    total_cost = 0
    print(env.render())
    print(f"Timestep: {1}")
    print(f"State: {env.get_state()}")
    print(f"Action: {None}")
    print(f"Cost: {0}")
    time.sleep(1)

    for i, action in enumerate(actions):
      state, cost, terminated = env.step(action)
      total_cost += cost
      clear_output(wait=True)

      print(env.render())
      print(f"Timestep: {i + 2}")
      print(f"State: {state}")
      print(f"Action: {action}")
      print(f"Cost: {cost}")
      print(f"Total cost: {total_cost}")
      
      time.sleep(1)

      if terminated is True:
        break

## 1. BFS-G
**TO DO:** implement Breadth First Search (BFS) algorithm on graph like shown in class.


Now lets test your BFS agent!

In [19]:
env = DragonBallEnv(MAPS["8x8"])
BFS_agent = BFSAgent()
actions, total_cost, expanded = BFS_agent.search(env)
print(f"Total_cost: {total_cost}")
print(f"Expanded: {expanded}")
print(f"Actions: {actions}")

assert total_cost == 119.0, "Error in total cost returned"

Total_cost: 119.0
Expanded: 123
Actions: [0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 1, 1, 0, 0, 1, 1]


In [20]:
# print_solution(actions, env)

# Heapdict
For the next algorithms, you will be required to maintain an "open" queue based on a certain value (g/h/v). To manage these queues efficiently and conveniently, please use [Heapdict](https://www.geeksforgeeks.org/priority-queue-using-queue-and-heapdict-module-in-python/). Heapdict implements the MutableMapping ABC, meaning it works pretty much like a regular Python [dictionary](https://www.geeksforgeeks.org/python-dictionary/). It’s designed to be used as a priority queue. Along with functions provided by ordinary dict(), it also has popitem() and peekitem() functions which return the pair with the lowest priority.

Note:

When two nodes have the same minimum value, select the node with the lower state index first. Instead of defining priority as an integer, you can define it as a tuple (value, state, ...).

## 2. Weighted A*
TO DO: implement Wighted A* like shown in class.

Note:

*   A parameter called `h_weight` is passed to `Greedy_Best_First_search()`, which indicates how much weight is given to the heuristics (ranging from 0 to 1).
*   The heurisitcs needed to be implemented. Instructions in dry pdf.






In [21]:
env = DragonBallEnv(MAPS["8x8"])
WA_agent = WeightedAStarAgent()
actions, total_cost, expanded = WA_agent.search(env, h_weight=0.5)
print(f"Total_cost: {total_cost}")
print(f"Expanded: {expanded}")
print(f"Actions: {actions}")

# w2_actions, w2_cost, w2_expanded = WA_agent.search(env, h_weight = 1)
# print(f"Total_cost: {w2_cost}")
# print(f"Expanded: {w2_expanded}")
# print(f"Actions: {w2_actions}")

expanded: (0, False, False) f-value:  3.0 

expanded: (8, False, False) f-value:  7.5 

expanded: (1, False, False) f-value:  8.5 

expanded: (16, False, False) f-value:  8.5 

expanded: (9, False, False) f-value:  13.0 

expanded: (24, False, False) f-value:  13.0 

expanded: (2, False, False) f-value:  14.0 

expanded: (17, False, False) f-value:  14.0 

expanded: (32, False, False) f-value:  17.5 

expanded: (10, False, False) f-value:  18.5 

expanded: (25, False, False) f-value:  18.5 

expanded: (3, False, False) f-value:  19.0 

expanded: (33, False, False) f-value:  19.0 

expanded: (18, False, False) f-value:  19.5 

expanded: (40, False, False) f-value:  22.0 

expanded: (4, False, False) f-value:  23.5 

expanded: (11, False, False) f-value:  23.5 

expanded: (26, False, False) f-value:  24.0 

expanded: (34, False, False) f-value:  24.5 

expanded: (48, True, False) f-value:  24.5 

expanded: (5, False, False) f-value:  28.0 

expanded: (12, False, False) f-value:  28.0 

e

In [22]:
# print_solution(actions, env)

## 3. A*-epsilon::
TO DO: implement A*-epsilon: like shown in class.

use the same heuristic as in previous sections.

Note:
*   A parameter called `epsilon` is passed to `A_star_epsilon_search()`.
*   We will not test the amount of expanded nodes for this algorithm.


In [23]:
AStarEpsilon_agent = AStarEpsilonAgent()
actions, total_cost, expanded = AStarEpsilon_agent.search(env, epsilon=100)
print(f"Total_cost: {total_cost}")
print(f"Expanded: {expanded}")
print(f"Actions: {actions}")

expanded: (0, False, False) f-value:  3.0 

expanded: (1, False, False) f-value:  8.5 

expanded: (8, False, False) f-value:  7.5 

expanded: (16, False, False) f-value:  8.5 

expanded: (2, False, False) f-value:  14.0 

expanded: (9, False, False) f-value:  13.0 

expanded: (17, False, False) f-value:  14.0 

expanded: (24, False, False) f-value:  13.0 

expanded: (3, False, False) f-value:  19.0 

expanded: (10, False, False) f-value:  18.5 

expanded: (18, False, False) f-value:  19.5 

expanded: (25, False, False) f-value:  18.5 

expanded: (32, False, False) f-value:  17.5 

expanded: (33, False, False) f-value:  19.0 

expanded: (4, False, False) f-value:  23.5 

expanded: (11, False, False) f-value:  23.5 

expanded: (26, False, False) f-value:  24.0 

expanded: (40, False, False) f-value:  22.0 

expanded: (48, True, False) f-value:  24.5 

expanded: (34, False, False) f-value:  24.5 

expanded: (5, False, False) f-value:  28.0 

expanded: (12, False, False) f-value:  28.0 

e

In [24]:
# print_solution(actions, env)

## 7. Benchmarking:
In this section we want to compare the different search algorithms. The take-home message is that there is no "one algorithm fits all".

In [25]:
import csv

test_boards = {
"map12x12": 
['SFAFTFFTHHHF',
'AFLTFFFFTALF',
'LHHLLHHLFTHD',
'HALTHAHHADHF',
'FFFTFHFFAHFL',
'LLTHFFFAHFAT',
'HAAFFALHTATF',
'LLLFHFFHTLFH',
'FATAFHTTFFAF',
'HHFLHALLFTLF',
'FFAFFTTAFAAL',
'TAAFFFHAFHFG'],
"map15x15": 
['SFTTFFHHHHLFATF',
'ALHTLHFTLLFTHHF',
'FTTFHHHAHHFAHTF',
'LFHTFTALTAAFLLH',
'FTFFAFLFFLFHTFF',
'LTAFTHFLHTHHLLA',
'TFFFAHHFFAHHHFF',
'TTFFLFHAHFFTLFD',
'TFHLHTFFHAAHFHF',
'HHAATLHFFLFFHLH',
'FLFHHAALLHLHHAT',
'TLHFFLTHFTTFTTF',
'AFLTDAFTLHFHFFF',
'FFTFHFLTAFLHTLA',
'HTFATLTFHLFHFAG'],
"map20x20" : 
['SFFLHFHTALHLFATAHTHT',
'HFTTLLAHFTAFAAHHTLFH',
'HHTFFFHAFFFFAFFTHHHT',
'TTAFHTFHTHHLAHHAALLF',
'HLALHFFTHAHHAFFLFHTF',
'AFTAFTFLFTTTFTLLTHDF',
'LFHFFAAHFLHAHHFHFALA',
'AFTFFLTFLFTAFFLTFAHH',
'HTTLFTHLTFAFFLAFHFTF',
'LLALFHFAHFAALHFTFHTF',
'LFFFAAFLFFFFHFLFFAFH',
'THHTTFAFLATFATFTHLLL',
'HHHAFFFATLLALFAHTHLL',
'HLFFFFHFFLAAFTFFDAFH',
'HTLFTHFFLTHLHHLHFTFH',
'AFTTLHLFFLHTFFAHLAFT',
'HAATLHFFFHHHHAFFFHLH',
'FHFLLLFHLFFLFTFFHAFL',
'LHTFLTLTFATFAFAFHAAF',
'FTFFFFFLFTHFTFLTLHFG']}

test_envs = {}
for board_name, board in test_boards.items():
    test_envs[board_name] = DragonBallEnv(board)


BFS_agent = BFSAgent()
WAStar_agent = WeightedAStarAgent()

weights = [0.5, 0.7, 0.9]

agents_search_function = [
    BFS_agent.search,
]

header = ['map',  "BFS-G cost",  "BFS-G expanded",\
           'WA* (0.5) cost', 'WA* (0.5) expanded', 'WA* (0.7) cost', 'WA* (0.7) expanded', 'WA* (0.9) cost', 'WA* (0.9) expanded']

with open("results.csv", 'w') as f:
  writer = csv.writer(f)
  writer.writerow(header)
  for env_name, env in test_envs.items():
    data = [env_name]
    for agent in agents_search_function:
      _, total_cost, expanded = agent(env)
      data += [total_cost, expanded]
    for w in weights:
        _, total_cost, expanded = WAStar_agent.search(env, w)
        data += [total_cost, expanded]

    writer.writerow(data)

expanded: (0, False, False) f-value:  6.0 

expanded: (12, False, False) f-value:  6.5 

expanded: (24, False, False) f-value:  6.5 

expanded: (1, False, False) f-value:  10.5 

expanded: (2, False, False) f-value:  11.0 

expanded: (13, False, False) f-value:  11.0 

expanded: (14, False, False) f-value:  11.0 

expanded: (15, False, False) f-value:  12.0 

expanded: (27, False, False) f-value:  12.0 

expanded: (28, False, False) f-value:  12.0 

expanded: (39, False, False) f-value:  13.0 

expanded: (38, False, False) f-value:  14.0 

expanded: (51, False, False) f-value:  15.0 

expanded: (3, False, False) f-value:  15.5 

expanded: (37, False, False) f-value:  15.5 

expanded: (4, False, False) f-value:  16.5 

expanded: (16, False, False) f-value:  16.5 

expanded: (50, False, False) f-value:  19.5 

expanded: (52, False, False) f-value:  19.5 

expanded: (5, False, False) f-value:  21.0 

expanded: (17, False, False) f-value:  21.0 

expanded: (49, False, False) f-value:  21.0